In [1]:
%config Completer.use_jedi = False

# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
# !pip install pycm

In [3]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from helper import *
import keras as keras
from pycm import *
from sklearn.metrics import classification_report, confusion_matrix

In [14]:
X_test = np.load('Data/test_data.npy')
X_train = np.load('Data/train_data.npy')

y_train = X_train[:,-1]
X_train = X_train[:,:-1]

y_test = X_test[:,-1]
X_test = X_test[:,:-1]

print (y_test.shape, X_test.shape, y_train.shape, X_train.shape)

X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],1)
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

print (y_test.shape, X_test.shape, y_train.shape, X_train.shape)

(203187,) (203187, 186) (204775,) (204775, 186)
(203187, 3) (203187, 186, 1) (204775, 3) (204775, 186, 1)


In [8]:
X_Gen = np.vstack((X_train, X_test))
y_Gen = np.hstack((y_train, y_test))

In [9]:
X_Gen.shape

(407962, 186)

In [10]:
y_Gen.shape

(407962,)

In [5]:
# input1 = Input(shape=(186,1), name='input')
# x = Conv1D(filters=32, kernel_size=3, padding='same', strides=1, kernel_initializer='he_normal', activation='relu')(input1)
# x = Flatten()(x)
# # x = Dense(3)(x)
# out = Dense(3, activation='softmax')(x)
# model = Model(inputs=input1, outputs=out)
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

In [15]:
WINDOW_SIZE = 186
INPUT_FEAT = 1
OUTPUT_CLASS = 3
cb = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=5)]

# earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
# mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
# reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')

# ModelCheckpoint(filepath=str(cvset)+'/check_model.h5', monitor='val_loss', save_best_only=True),
        
# model.fit(Xtr_more, Ytr_more, batch_size=batch_size, epochs=50, verbose=0, callbacks=[earlyStopping, mcp_save, reduce_lr_loss], validation_split=0.25)


In [16]:
for LAYER in range(1,8):
    filename = 'ResNet_' + str(LAYER)
    print (filename)
    if not os.path.exists(str(filename)):
        os.makedirs(str(filename))
    
    model = ResNet_model(WINDOW_SIZE,INPUT_FEAT,OUTPUT_CLASS,LAYER)
#     model.summary()

    history = model.fit(X_train, y_train, batch_size=32, epochs=1, callbacks=cb, validation_split=0.2, verbose=1)
    prediction(model, X_test, y_test, filename)
    save_the_model(filename, model)
    write_history(filename, history.history)

ResNet_1
6350/6350 [==============================] - 27s 4ms/step
Saved model to disk
ResNet_2
6350/6350 [==============================] - 30s 5ms/step
Saved model to disk
ResNet_3
6350/6350 [==============================] - 33s 5ms/step
Saved model to disk
ResNet_4
6350/6350 [==============================] - 39s 6ms/step
Saved model to disk
ResNet_5
6350/6350 [==============================] - 44s 7ms/step
Saved model to disk
ResNet_6
6350/6350 [==============================] - 51s 8ms/step
Saved model to disk
ResNet_7
6350/6350 [==============================] - 54s 9ms/step
Saved model to disk


In [ ]:
# keras.callbacks.ModelCheckpoint(filepath=str(filename)+'/check_model.hdf5', monitor='val_loss', save_best_only=True)
# for LAYER in range(1,8):
#     filename = 'pretrained_models/CNN_' + str(LAYER)
#     print (filename)
#     model = CNN_model(WINDOW_SIZE,INPUT_FEAT,OUTPUT_CLASS,LAYER)
#     model.load_weights(str(filename)+'/check_model.hdf5')
#     model.predict(X_train)
#     break

In [ ]:
# from sklearn.model_selection import StratifiedKFold
# kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
# # enumerate the splits and summarize the distributions
# for train_ix, test_ix in kfold.split(input_files, np.argmax(final_labels, axis=1)):
#     X_train = input_files[train_ix]
#     y_train = final_labels[train_ix]
#     X_val = input_files[test_ix]
#     y_val = final_labels[test_ix]
#     print (cvset)
# #     print (X_train, y_train, X_val, y_val)
#     if not os.path.exists(str(cvset)):
#         os.makedirs(str(cvset))
    
#     model = ResNet_model(2500,12,27,8)
#     batch_size = 16

#     # train_generator = generator(input_files, final_labels, batch_size)
#     train_generator = generator(input_directory, X_train, y_train, batch_size)
#     val_generator = generator(input_directory, X_val, y_val, batch_size)

#     train_samples=np.ceil(len(X_train) / batch_size)
#     val_samples=np.ceil(len(X_val) / batch_size)

#     callbacks = [
#         ModelCheckpoint(filepath=str(cvset)+'/check_model.h5', monitor='val_loss', save_best_only=True),
#         EarlyStopping(monitor='val_acc', verbose=1, patience=5)]

#     # ModelCheckpoint('weights-best_k{}_r{}.hdf5'.format(k,r), monitor='val_loss', save_best_only=True, verbose=1)

#     history = model.fit_generator(train_generator, steps_per_epoch=train_samples, epochs=100, verbose=1, 
#                         validation_data=val_generator, validation_steps=val_samples, callbacks=callbacks)

#     save_the_model(cvset, model)
#     write_history(cvset, history.history)
#     cvset = cvset+1